# Making it pretty using Vuetify components and ipyvuetify



In [ ]:
from typing import List
from download_listing import get_listing
from mortgage import mortgage
from home_value import home_value, letting_cash_flow
from charts import altair_outcomes_chart, altair_wealth_chart, altair_format
from charts import bokeh_outcomes_chart, bokeh_wealth_chart, show

### Generate an investment scenario

In [ ]:
import pandas as pd
import numpy as np

def scenario(price=500_000, transaction_cost = 0.05, downpayment = 0.05,
             years = 20, inflation = .018, inflation_houseprice = 0.045,
             rate_mortgage = 0.028, rental_rate = 0, interest_only = False):
    
    # Report input assumptions
    scenario_conditions = [
        f'House price: £{price}',
        f'transaction_cost: £{transaction_cost*price:.0f} ({100*transaction_cost:.2f}%)',
        f'downpayment: £{downpayment*price:.0f} ({100*downpayment:.2f}%)',
        f'term (years): {years}',
        f'inflation {100*inflation:.2f}%',
        f'house price appreciation: {100*inflation_houseprice:.2f}%',
        f'rate_mortgage: {100*rate_mortgage:.2f}%',
        f'monthly rental income: £{rental_rate:.2f}',
        f'interest-only: {"yes" if interest_only else "no"}'
    ]
    transaction_cost *= price
    downpayment *= price 

    # Schedule and outcomes over time
    loan_schedule = mortgage(price, downpayment, rate_mortgage, years, interest_only=interest_only)
    debt_balance = loan_schedule.debt_balance.resample('Y').last()
    interest_paid = loan_schedule.interest_payments.resample('Y').sum().cumsum()

    # Home value and letting income
    home = home_value(price, inflation_houseprice, years)
    letting_income = letting_cash_flow(rental_rate, years, inflation).cumsum()

    # Actual cash flow
    cash_flow = (
        loan_schedule.full_payments.resample('Y').sum()
        + letting_income
    )
    cash_flow.iloc[0] -= transaction_cost
    cash_flow.iloc[0] -= downpayment
    cash_flow = cash_flow.cumsum()

    # Real situation
    wealth = home + letting_income - downpayment - debt_balance - transaction_cost

    # Discounted value at time T - discounting by inflation
    df = pd.DataFrame(
        data = [debt_balance, interest_paid, cash_flow, wealth],
        index = 'debt_balance, interest_payments, cash_flow, wealth'.split(', ')
    ).T.dropna()

    discounting_factor = np.pv(inflation, np.arange(len(df)), 0, fv=-1)
    df = df.mul(discounting_factor, axis=0)
    
    # Charts
    adf = altair_format(df)
    outcomes_c = altair_outcomes_chart(adf)
    wealth_c = altair_wealth_chart(adf)
    
    return df, scenario_conditions, outcomes_c, wealth_c

## Trying Vuetify

In [ ]:
from IPython.display import display, clear_output
import ipywidgets
import ipyvuetify as v
import traitlets
from traitlets import Int, Float, Unicode, Bool

In [ ]:
v.*?

In [ ]:
v.Slider(v_model=55, min=10, max=95, step=5, thumb_label='always', class_='ma-5 pa-5')

In [ ]:
v.Checkbox(v_model=True, label='Hi there')

In [ ]:
v.Col(md='6', children=[
           v.Slider(v_model=100000, min=10, max=100000, step=5, thumb_label='always', class_='ma-5 pa-5')
])

# Creating the app

## Model

In [ ]:
class Model(traitlets.HasTraits):
    price = Int(500_000)
    transaction_cost = Float(0.05)
    downpayment = Float(0.05)
    years = Int(20)
    inflation = Float(.018)
    inflation_houseprice = Float(0.045)
    rate_mortgage = Float(0.028)
    rental_rate = Int(0)
    interest_only = Bool(False)
    
model = Model()

## View

### Inputs

In [ ]:
price_slider = v.Slider(v_model=250_000, 
                        min=50_000, max=1_000_000, 
                        step=25_000
                       )
price_field = v.TextField(v_model=250_000, outlined=True, dense=True, prefix='£')


term_slider = v.Slider(v_model=20, min=5, max=30, step=5, thumb_label='always')

interest_only_checkbox = v.Checkbox(v_model=False, label='interest-only loan')

inflation_slider = v.Slider(v_model=0.02, min=0.0, max=0.15, step=0.0025, thumb_label='always')

inflation_h_slider = v.Slider(v_model=0.02, min=-0.1, max=0.25, step=0.01, thumb_label='always')

### Layout elements

In [ ]:
def make_card(title="Title", children=[]):
    card_content = v.CardText(children=children)
    return (
        v.Card(outlined=True, tile=True, #class_='ma-1 pa-2', 
            children=[
                v.CardTitle(children=[title]),
                card_content
            ]),
        card_content
    )

def make_control(title, children):
    return (
        v.Card(flat=True, tile=True, #class_='ma-1 pa-2', 
            children=[
                v.CardText(children=[title]),
                v.CardText(children=children)
            ])
        )

# To be used in the presentation section
def make_infobox(scenario_conditions: List[str]):
    return [
        v.List(dense=True, children=[
            v.ListItem(children=[line])
            for line in scenario_conditions
        ])
    ]

controls_box, cbc = make_card('Controls', children=[
    make_control('Price', [price_slider, price_field]),
    make_control('Term', [term_slider]),
    make_control('Inflation', [inflation_slider]),
    make_control('Home appreciation', [inflation_h_slider]),
    make_control('Interest-only', [interest_only_checkbox]),
])
conditions_infobox, cic = make_card('Scenario assumptions')
wealth_chart_box, wcc = make_card('Wealth over time', [ipywidgets.Output()])
outcomes_chart_box, obc = make_card('Financial outcomes', [ipywidgets.Output()])

### Layout arrangement

In [ ]:
center_content = v.Container(fluid=True, children=[
    v.Row(dense=True, children=[
        v.Col(cols="12", children=[
            v.Row(align='start', justify='left', dense=True, children=[
                v.Col(cols="9", md="9", children=[
                    wealth_chart_box
                ]),
                v.Col(cols="9", md="9", children=[
                    outcomes_chart_box
                ])
            ])
        ]),
        v.Col(cols="12", children=[
            v.Row(align='start', justify='left', dense=True, children=[
#                 v.Col(cols="6", md="6", children=[
#                     controls_box
#                 ]),
                v.Col(cols="9", md="9", children=[
                    conditions_infobox
                ])
            ])
        ])
    ])
])
navbar = v.NavigationDrawer(absolute=True, permanent=True, right=True, raised=True, 
                            class_='ma-3 pa-6', children=[
      controls_box
])

app = v.Container(_metadata={'mount_id': 'content-main'}, children=[
      center_content, navbar
])

## Presentation

### On-refresh

In [ ]:
def render_scenario(change):
    with model.hold_trait_notifications():
        (
             schedule_df, 
             scenario_conditions, 
             outcomes_c, 
             wealth_c
        ) = scenario(
                price                = model.price,
                downpayment          = model.downpayment,
                inflation            = model.inflation,
                inflation_houseprice = model.inflation_houseprice,
                interest_only        = model.interest_only,
                rate_mortgage        = model.rate_mortgage,
                rental_rate          = model.rental_rate,
                transaction_cost     = model.transaction_cost,
                years                = model.years)

        cic.children = make_infobox(scenario_conditions)

        with obc.children[0]:
            clear_output(wait=True)
            display(outcomes_c)

        with wcc.children[0]:
            clear_output(wait=True)
            display(wealth_c)
    
model.observe(render_scenario, traitlets.All)

### Link inputs to model

In [ ]:
ipywidgets.jslink((price_slider, 'v_model'), (price_field, 'v_model'))

traitlets.link(
    (model, 'price'), (price_slider, 'v_model')
)
traitlets.link(
    (model, 'years'), (term_slider, 'v_model')
)
traitlets.link(
    (model, 'inflation'), (inflation_slider, 'v_model')
)
traitlets.link(
    (model, 'inflation_houseprice'), (inflation_h_slider, 'v_model')
)
traitlets.link(
    (model, 'interest_only'), (interest_only_checkbox, 'v_model')
);

## Show the app

In [ ]:
render_scenario('')

app

## Exercise: Add a slider!

In [ ]:
model.trait_names()